In [368]:
# from mido import MidiFile
import globalvar as GlobalVar
from funcMidi import * # create by SH
# MidiFileName = 'p20190612_midi_test_mix.mid'
# MidiFileName = 'p20190716_midi_test.mid'
# MidiFileName = 'p20190818 mix from TMD.mid'
# MidiFileName = 'Fleck, Bela - Daybreak.mid'
# MidiFileName = 'Aguado_12valses_Op1_No1.mid'
MidiFileName = 'Aguado_12valses_Op1_No1_downloadmidi.mid'

# midiArray = MidiCategorize(MidiFileName)
(midiArray, dicEvent) = MidiCategorize(MidiFileName)
dicNoteOnFingerBoard = funcCreatNoteDic()
midiArray

Track 0: 
capo = 0
tuning = [64, 59, 55, 50, 45, 40]
Notes in Finger board:
[64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76]
[59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]
[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67]
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62]
[45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57]
[40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]


[[57, 0, 96, 'MIX', 0],
 [45, 0, 384, 'MIX', 0],
 [61, 96, 128, 'MIX', 0],
 [64, 128, 256, 'MIX', 0],
 [69, 256, 384, 'MIX', 0],
 [73, 384, 480, 'MIX', 1],
 [45, 384, 768, 'MIX', 1],
 [69, 480, 512, 'MIX', 1],
 [64, 512, 640, 'MIX', 1],
 [61, 640, 768, 'MIX', 1],
 [64, 768, 864, 'MIX', 2],
 [40, 768, 1152, 'MIX', 2],
 [62, 864, 896, 'MIX', 2],
 [59, 896, 1024, 'MIX', 2],
 [56, 1024, 1152, 'MIX', 2],
 [57, 1152, 1248, 'MIX', 3],
 [45, 1152, 1408, 'MIX', 3],
 [61, 1248, 1280, 'MIX', 3],
 [52, 1280, 1536, 'MIX', 3],
 [40, 1408, 1536, 'MIX', 3],
 [57, 1536, 1632, 'MIX', 4],
 [45, 1536, 1920, 'MIX', 4],
 [61, 1632, 1664, 'MIX', 4],
 [64, 1664, 1792, 'MIX', 4],
 [69, 1792, 1920, 'MIX', 4],
 [68, 1920, 2016, 'MIX', 5],
 [40, 1920, 2304, 'MIX', 5],
 [71, 2016, 2048, 'MIX', 5],
 [76, 2048, 2176, 'MIX', 5],
 [64, 2176, 2304, 'MIX', 5],
 [63, 2304, 2400, 'MIX', 6],
 [47, 2304, 2432, 'MIX', 6],
 [57, 2304, 2432, 'MIX', 6],
 [66, 2400, 2432, 'MIX', 6],
 [71, 2432, 2560, 'MEL', 7],
 [63, 2560, 2688,

In [176]:
# debug
# events[59]
midiArray = [
    [59, 15488, 15616, 'CHO', 58],
 [62, 15488, 15616, 'CHO', 58],
 [45, 15488, 15616, 'CHO', 58],
 [57, 15616, 15872, 'MIX', 59],
 [61, 15616, 15872, 'MIX', 59],
 [45, 15616, 15744, 'MIX', 59],
 [45, 15744, 15872, 'MIX', 59]
]

In [181]:
# debug
midiArrayOrder = copy.deepcopy(midiArray)
for i in range(len(midiArrayOrder)):
    midiArrayOrder[i].append(i)
midiArrayOrder

[[57, 0, 96, 'MIX', 0, 0],
 [45, 0, 384, 'MIX', 0, 1],
 [61, 96, 128, 'MIX', 0, 2],
 [64, 128, 256, 'MIX', 0, 3],
 [69, 256, 384, 'MIX', 0, 4],
 [73, 384, 480, 'MIX', 1, 5],
 [45, 384, 768, 'MIX', 1, 6],
 [69, 480, 512, 'MIX', 1, 7],
 [64, 512, 640, 'MIX', 1, 8],
 [61, 640, 768, 'MIX', 1, 9],
 [64, 768, 864, 'MIX', 2, 10],
 [40, 768, 1152, 'MIX', 2, 11],
 [62, 864, 896, 'MIX', 2, 12],
 [59, 896, 1024, 'MIX', 2, 13],
 [56, 1024, 1152, 'MIX', 2, 14],
 [57, 1152, 1248, 'MIX', 3, 15],
 [45, 1152, 1408, 'MIX', 3, 16],
 [61, 1248, 1280, 'MIX', 3, 17],
 [52, 1280, 1536, 'MIX', 3, 18],
 [40, 1408, 1536, 'MIX', 3, 19],
 [57, 1536, 1632, 'MIX', 4, 20],
 [45, 1536, 1920, 'MIX', 4, 21],
 [61, 1632, 1664, 'MIX', 4, 22],
 [64, 1664, 1792, 'MIX', 4, 23],
 [69, 1792, 1920, 'MIX', 4, 24],
 [68, 1920, 2016, 'MIX', 5, 25],
 [40, 1920, 2304, 'MIX', 5, 26],
 [71, 2016, 2048, 'MIX', 5, 27],
 [76, 2048, 2176, 'MIX', 5, 28],
 [64, 2176, 2304, 'MIX', 5, 29],
 [63, 2304, 2400, 'MIX', 6, 30],
 [47, 2304, 2432, '

In [370]:
# find Best Path 整理包
# 跑這隻之前 要把後面的func都跑過一次, debug系列除外
# calEventNum = 59

# 找到inf bug所在點--> calEventNum = 59, fromNumber = 216

# events, uniqTime = funcMidiEvents(midiArray, calEventNum = 59, fromNumber = 216)
events, uniqTime = funcMidiEvents(midiArray)
eventsNotes = funcArrangeEventNotes(events)
possible = funcNotes2Possibles(eventsNotes)
costBest = funcBestPath(eventsNotes, possible, printCostChord = False, printCostTran = False)
solutions, bestSolution = funcSolutions(costBest, possible)



In [371]:
# costBest[-1]
bestSolution

['cost =',
 5.148748123734997e+80,
 [[[3, 2, 1], [5, 0, 0]],
  [[5, 0, 0], [2, 2, 2]],
  [[5, 0, 0], [1, 0, 0]],
  [[5, 0, 0], [1, 5, 1]],
  [[1, 9, 3], [5, 0, 0]],
  [[5, 0, 0], [1, 5, 3]],
  [[5, 0, 0], [1, 0, 0]],
  [[5, 0, 0], [2, 2, 2]],
  [[1, 0, 0], [6, 0, 0]],
  [[6, 0, 0], [2, 3, 2]],
  [[6, 0, 0], [2, 0, 0]],
  [[6, 0, 0], [3, 1, 1]],
  [[3, 2, 2], [5, 0, 0]],
  [[5, 0, 0], [2, 2, 1]],
  [[5, 0, 0], [4, 2, 2]],
  [[4, 2, 2], [6, 0, 0]],
  [[3, 2, 1], [5, 0, 0]],
  [[5, 0, 0], [2, 2, 2]],
  [[5, 0, 0], [1, 0, 0]],
  [[5, 0, 0], [1, 5, 2]],
  [[1, 4, 1], [6, 0, 0]],
  [[6, 0, 0], [1, 7, 1]],
  [[6, 0, 0], [1, 12, 1]],
  [[6, 0, 0], [1, 0, 0]],
  [[2, 4, 4], [5, 2, 1], [3, 2, 2]],
  [[5, 2, 1], [3, 2, 2], [1, 2, 3]],
  [[1, 7, 3]],
  [[2, 4, 2], [3, 2, 1]],
  [[1, 0, 0], [4, 2, 3], [3, 1, 1]],
  [[3, 2, 2], [5, 0, 0]],
  [[5, 0, 0], [2, 2, 1]],
  [[5, 0, 0], [1, 0, 0]],
  [[5, 0, 0], [1, 5, 1]],
  [[1, 9, 3], [5, 0, 0]],
  [[5, 0, 0], [1, 5, 3]],
  [[5, 0, 0], [1, 0, 0]],
  [[5,

In [297]:
bestSolution
eventsNotes
print(eventsNotes)
names = []
for event in eventsNotes:
    absEvent = list(map(abs, event))
    nameTemp = dicNameNotes(absEvent)
    names.append(nameTemp)
print(names)

possible
newpossible = [[possible[0][1]]]
newpossible.append([possible[1][0]])
newpossible.append([possible[2][36]])
newpossible.append([possible[3][36]])
newpossible.append([possible[4][14]])
newpossible.append([possible[5][11]])
newpossible.append([possible[6][11]])
# possible[6]
newpossible
len(possible)

[[57, 61, 52], [-52, 56, 59], [61, 64, 45], [-61, -64, 45], [59, 62, 45], [57, 61, 45], [-57, -61, 45]]
[['A3', 'C#4', 'E3'], ['E3', 'G#3', 'B3'], ['C#4', 'E4', 'A2'], ['C#4', 'E4', 'A2'], ['B3', 'D4', 'A2'], ['A3', 'C#4', 'A2'], ['A3', 'C#4', 'A2']]


7

In [299]:
# eventsNotes = 
costBest = funcBestPath(eventsNotes, newpossible, printCostChord = True, printCostTran = True)
solutions, bestSolution = funcSolutions(costBest, newpossible)
# eventsNotes

number = 0
printCostChord =
[[47.]]
printCostTran =
[[3.]]
printCostAll =
[[50.]]

number = 1
printCostChord =
[[43.]]
printCostTran =
[[3.]]
printCostAll =
[[46.]]

number = 2
printCostChord =
[[43.]]
printCostTran =
[[0.]]
printCostAll =
[[43.]]

number = 3
printCostChord =
[[53.]]
printCostTran =
[[5.]]
printCostAll =
[[57.]]

npMinValue = [57.47638889]
npMinIndex = [0]
printCostChord =
[[53.]]
printCostTran =
[[5.]]
printCostAll =
[[57.]]

number = 4
printCostChord =
[[53.]]
printCostTran =
[[7.]]
printCostAll =
[[60.]]

number = 5
printCostChord =
[[43.]]
printCostTran =
[[0.]]
printCostAll =
[[44.]]



In [163]:
loopSolution = []
for calEventNum in range(60,70):
    print(calEventNum)
    events, uniqTime = funcMidiEvents(midiArray, calEventNum)
    eventsNotes = funcArrangeEventNotes(events)
    possible = funcNotes2Possibles(eventsNotes)
    costBest = funcBestPath(eventsNotes, possible)
    solutions, bestSolution = funcSolutions(costBest, possible)
    
    title = ['eventNum='+str(calEventNum)]
    title.append(bestSolution)
    loopSolution.append(title)
    
    print("eventNum=" + str(calEventNum) + ", cost=" + str(bestSolution[1]))

# bestSolution
# events

55
eventNum=55, cost=6407.806646825396
56
eventNum=56, cost=6513.169146825396
57
eventNum=57, cost=6602.419146825396
58
eventNum=58, cost=6671.739980158729
59
eventNum=59, cost=inf


In [172]:
# debug
eventsNotes = [[57, 61, 45], [-57, -61, 45]]
possible = funcNotes2Possibles(eventsNotes)
costBest = funcBestPath(eventsNotes, possible, printOut = True)

0
[[ 1. inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
  inf inf]
 [inf  2.  1. inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
  inf inf]
 [inf  2.  1. inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
  inf inf]
 [inf inf inf  2.  1. inf inf inf inf inf inf inf inf inf inf inf inf inf
  inf inf]
 [inf inf inf  2.  0. inf inf inf inf inf inf inf inf inf inf inf inf inf
  inf inf]
 [inf inf inf inf inf  3.  3.  1. inf inf inf inf inf inf inf inf inf inf
  inf inf]
 [inf inf inf inf inf  3.  2.  1. inf inf inf inf inf inf inf inf inf inf
  inf inf]
 [inf inf inf inf inf  2.  2.  0. inf inf inf inf inf inf inf inf inf inf
  inf inf]
 [inf inf inf inf inf inf inf inf  0. inf inf inf inf inf inf inf inf inf
  inf inf]
 [inf inf inf inf inf inf inf inf inf  0. inf inf inf inf inf inf inf inf
  inf inf]
 [inf inf inf inf inf inf inf inf inf inf  0. inf inf inf inf inf inf inf
  inf inf]
 [inf inf inf inf inf inf inf inf inf inf inf  0. inf inf inf i

In [170]:
bestSolution
# title = ['eventNum='+str(calEventNum)]
# title.append(bestSolution)
# loopSolution.append(title)
eventsNotes

In [238]:
def dicNameNotes(notes):
    names = []
    for note in notes:
        modNote = note % 12
        if modNote == 0:
            names.append('C'+str(note//12))
        elif modNote ==1:
            names.append('C#'+str(note//12 -1))
        elif modNote ==2:
            names.append('D'+str(note//12 -1))
        elif modNote ==3:
            names.append('D#'+str(note//12 -1))
        elif modNote ==4:
            names.append('E'+str(note//12 -1))
        elif modNote ==5:
            names.append('F'+str(note//12 -1))
        elif modNote ==6:
            names.append('F#'+str(note//12 -1))
        elif modNote ==7:
            names.append('G'+str(note//12 -1))
        elif modNote ==8:
            names.append('G#'+str(note//12 -1))
        elif modNote ==9:
            names.append('A'+str(note//12 -1))
        elif modNote ==10:
            names.append('A#'+str(note//12 -1))
        elif modNote ==11:
            names.append('B'+str(note//12 -1))
    return names

dicNameNotes([45, 63])

['A2', 'D#4']

In [183]:
# list by event

def funcMidiEvents(midiArray, calEventNum = float("inf"), fromNumber = 0):
    events = []
    nn = fromNumber
    while nn < len(midiArray) and midiArray[nn][-1] <= calEventNum:
        if midiArray[nn][3] == 'MEL':
            events.append([midiArray[nn][:-1]])
            nn = nn + 1
        elif midiArray[nn][3] == 'MIX' or midiArray[nn][3] == 'CHO':
            subEvents = []
            currentEvent = midiArray[nn][4]
            while nn < len(midiArray) and midiArray[nn][4] == currentEvent and midiArray[nn][-1] <= calEventNum:
                subEvents.append(midiArray[nn][:-1])
                nn = nn + 1
            events.append(subEvents)
        else:
            break
            
    # unique time
    lastDMY = midiArray[-1][3]
    if lastDMY == 'MIX' or lastDMY == 'MEL' or lastDMY == 'CHO':
        allTime = [x[1:3] for x in midiArray]
    else:
        allTime = [x[1:3] for x in midiArray[:-1] ] # ignore dummy line
    flatAllTime = sum(allTime, [])
    uniqTime = list(set(flatAllTime))
    uniqTime.sort()
    
    return events, uniqTime
# events
# subEvents

In [105]:
# 依照時間 產生出note 包含tied note
def funcArrangeEventNotes(events):
    eventsNotes = []
    for event in events:
        if len(event) == 1:
            eventsNotes.append([event[0][0]])
        elif event[0][-1] =='CHO':
            choNotes = [x[0] for x in event]
            eventsNotes.append(choNotes)

        # most complex condition
        # made tied note to be Negative
        elif event[0][-1] == 'MIX':
            allTime = [x[1:3] for x in event]
            flatAllTime = sum(allTime, [])
            uniqTime = list(set(flatAllTime))
            uniqTime.sort()

            subEventNotes = []
            tempNotes = []
            for time in uniqTime[:-1]:

                for note in event:
                    if note[1] < time < note[2]:
                        try: 
                            tempNotes[tempNotes.index(note[0])] = -note[0]
                        except:
                            pass
                    elif note[1] == time:
                        tempNotes.append(note[0])
                    elif note[2] == time:
                        try:
                            tempNotes.remove(note[0])
                        except:
                            tempNotes.remove(-note[0])    
                subEventNotes.append(copy.deepcopy(tempNotes)) # should use deepcopy instead of copy
            for x in subEventNotes:
                eventsNotes.append(x) 
        
    return eventsNotes

In [27]:
eventsNotes[0] = [45, 57]

In [106]:
# run through events
# 有BUG存在!!!
# 目前len = 1的情況下 沒有去判斷延音
def funcNotes2Possibles(eventsNotes):
    possible =[]
    for choNote in eventsNotes:
        if len(choNote) == 1 :
            # BUG IS HERE ---------------------------------vvvvvvvvvvvvvv
            if choNote[0] < 0:
                print("BUG, no constraints for tied note when there is only one note")
            possible.append([ [x] for x in funcNote2Domain(abs(choNote[0]), dicNoteOnFingerBoard)])
    #         possible.append([funcNote2Domain(choNote[0], dicNoteOnFingerBoard)])
        else:
            subPossible = funcChordSolution(list(map(abs, choNote)), dicNoteOnFingerBoard)
            possible.append(subPossible)
    return possible

# 如果是從前一個就start, 還沒結束的NOte, 改為 -Note (如-55)
# 當作一個標記flag
# 算chord solution時要拿abs
# 可是算path的時候
# 有看到負號就只能抓前一組相同的note posibtion


In [12]:
# debug, 只選取少部分想觀察的possible
del possible[0][12:]
del possible[0][0:3]
del possible[1][18:]
del possible[1][0:5]
possible

[[[[6, 5, 1], [4, 7, 3]],
  [[6, 5, 1], [4, 7, 2]],
  [[6, 5, 1], [4, 7, 4]],
  [[5, 0, 0], [4, 7, 3]],
  [[5, 0, 0], [4, 7, 1]],
  [[5, 0, 0], [3, 2, 1]],
  [[5, 0, 0], [3, 2, 2]],
  [[5, 0, 0], [4, 7, 2]],
  [[5, 0, 0], [3, 2, 3]]],
 [[[6, 5, 1], [3, 6, 2]],
  [[6, 5, 1], [3, 6, 3]],
  [[5, 0, 0], [3, 6, 4]],
  [[5, 0, 0], [2, 2, 2]],
  [[5, 0, 0], [3, 6, 1]],
  [[5, 0, 0], [4, 11, 2]],
  [[5, 0, 0], [2, 2, 1]],
  [[5, 0, 0], [2, 2, 4]],
  [[5, 0, 0], [4, 11, 3]],
  [[5, 0, 0], [3, 6, 2]],
  [[5, 0, 0], [3, 6, 3]],
  [[5, 0, 0], [4, 11, 4]],
  [[5, 0, 0], [2, 2, 3]]],
 [[[6, 5, 2], [2, 5, 3]],
  [[6, 5, 2], [2, 5, 4]],
  [[6, 5, 2], [1, 0, 0]],
  [[6, 5, 4], [1, 0, 0]],
  [[6, 5, 1], [2, 5, 3]],
  [[6, 5, 1], [2, 5, 4]],
  [[6, 5, 1], [3, 9, 4]],
  [[6, 5, 1], [1, 0, 0]],
  [[6, 5, 1], [2, 5, 1]],
  [[6, 5, 1], [2, 5, 2]],
  [[5, 0, 0], [2, 5, 3]],
  [[5, 0, 0], [2, 5, 4]],
  [[5, 0, 0], [3, 9, 4]],
  [[5, 0, 0], [1, 0, 0]],
  [[5, 0, 0], [3, 9, 3]],
  [[5, 0, 0], [2, 5, 1]],
  [[5, 

In [8]:
# change global variable
GlobalVar.set_costChordWeight(4)
GlobalVar.get_costChordWeight()
GlobalVar.set_costFingerShiftSlide(3)
GlobalVar.get_costFingerShiftSlide()

3

In [356]:
npMinValue
np.shape(npMinValue)
newarray=np.zeros([len(npMinValue), 3])

for rwi in range(len(npMinValue)):
    newarray[rwi, :] = npMinValue[rwi]
newarray
# npMinValue
# np.shape(npCostMatrixUnit)
np.shape(newarray)[0]

21

In [369]:
# @@@@@@@@@ MIX 的版本 @@@@@@@@@
# 從MELODY 摳過來的
# 這邊不能合成function
# 因為要跟MEL 和 CHO 一起混著做
# 這邊只是確定funcCalMixCostMatrix 正確
# cost有先加previous再做運算的正確版


def funcBestPath(eventsNotes, possible, printCostChord = False, printCostTran = False):
    costBest = []

    iniSize = len(possible[0])
    npPreMinValue = np.zeros(iniSize)
    preMinIndex = [[0] for i in range(iniSize)]
    for i in range(len(possible)-1):
        event0Possible = possible[i]
        event1Possible = possible[i+1]
        mixNotes0 = eventsNotes[i]
        mixNotes1 = eventsNotes[i+1]
        # calculate MIX cost
        costMatrixUnitTran, costMatrixUnitChord = funcCalTranCostMatrix(event0Possible, event1Possible, mixNotes0, mixNotes1)    
        npCostMatrixUnitTran = np.array(costMatrixUnitTran)
        npCostMatrixUnitChord = np.array(costMatrixUnitChord)
        
        # add pre value to matrix
        npNewPreCostMatrix = np.zeros([len(npPreMinValue), np.shape(npCostMatrixUnitTran)[1]])
        for rwi in range(len(npPreMinValue)):
            npNewPreCostMatrix[rwi, :] = npPreMinValue[rwi]
        # total cost of this transition
        npCostMatrixUnit = npCostMatrixUnitChord + npCostMatrixUnitTran + npNewPreCostMatrix

        # print each costMatrixUnit independently, for debug
        if (printCostChord or printCostTran):
            # np print won't be limited
            np.set_printoptions(threshold=np.inf)
            print("number = " + str(i))
            if printCostChord:
                print("printCostChord =")
                print(np.around(npCostMatrixUnitChord, decimals = 0))
            if printCostTran:
                print("printCostTran =")
                print(np.around(npCostMatrixUnitTran, decimals = 0))
            print("printCostAll =")
            print(np.around(npCostMatrixUnit, decimals = 0))
            print("")
        
        # do matrix summary: minValue + minIndex
        npMinValue = np.min(npCostMatrixUnit, axis = 0)    
        npMinIndex = np.argmin(npCostMatrixUnit, axis = 0)
        minIndex = list(npMinIndex)

        # initialize for combining with previous event
        npCombMinValue = np.zeros(len(npMinValue))
        combMinIndex = [0 for icmi in range(len(npMinValue))]

        # minValue 
        for imin in range(len(npMinValue)):
            npCombMinValue[imin] = npMinValue[imin] + npPreMinValue[npMinIndex[imin]]
            combMinIndex[imin] = copy.deepcopy(preMinIndex[npMinIndex[imin]])
            combMinIndex[imin].append(copy.deepcopy(minIndex[imin]))
        npPreMinValue = npCombMinValue
        preMinIndex = combMinIndex

    #     minSummary = np.vstack
        costBest.append(copy.deepcopy([npCombMinValue,combMinIndex]))
    return costBest
# costBest[-1]

In [341]:
# 20190727 1930 bkp
# 為了把costTran + preCost 的bug修掉

# @@@@@@@@@ MIX 的版本 @@@@@@@@@
# 從MELODY 摳過來的
# 這邊不能合成function
# 因為要跟MEL 和 CHO 一起混著做
# 這邊只是確定funcCalMixCostMatrix 正確

def funcBestPath(eventsNotes, possible, printCostChord = False, printCostTran = False):
    costBest = []

    iniSize = len(possible[0])
    npPreMinValue = np.zeros(iniSize)
    preMinIndex = [[0] for i in range(iniSize)]
    # for i in range(2,3):
    # for i in range(1):
    for i in range(len(possible)-1):
        event0Possible = possible[i]
        event1Possible = possible[i+1]
        mixNotes0 = eventsNotes[i]
        mixNotes1 = eventsNotes[i+1]
        # calculate MIX cost
        costMatrixUnitTran, costMatrixUnitChord = funcCalTranCostMatrix(event0Possible, event1Possible, mixNotes0, mixNotes1)    
        npCostMatrixUnitTran = np.array(costMatrixUnitTran)
        npCostMatrixUnitChord = np.array(costMatrixUnitChord)
        npCostMatrixUnit = npCostMatrixUnitChord + npCostMatrixUnitTran

        # print each costMatrixUnit independently, for debug
        if (printCostChord or printCostTran):
            print("number = " + str(i))
            if printCostChord:
                print("printCostChord =")
                print(np.around(npCostMatrixUnitChord, decimals = 0))
            if printCostTran:
#                 print("printCostTran =")
#                 print(np.around(npCostMatrixUnitTran, decimals = 0))
                print("printCostAll =")
                print(np.around(npCostMatrixUnit, decimals = 0))
            print("")
        
        # do matrix summary: minValue + minIndex
        npMinValue = np.min(npCostMatrixUnit, axis = 0)    
        npMinIndex = np.argmin(npCostMatrixUnit, axis = 0)
        minIndex = list(npMinIndex)

        # initialize for combining with previous event
        npCombMinValue = np.zeros(len(npMinValue))
        combMinIndex = [0 for icmi in range(len(npMinValue))]
        
        # DEBUG USE
#         if i == 4:
        if True:
            print("npMinValue = " + str(npMinValue))
            print("npMinIndex = " + str(npMinIndex))
            print("npPreMinValue = " + str(npPreMinValue))
            print("preMinIndex = " + str(preMinIndex))
#             print("printCostChord =")
#             print(np.around(npCostMatrixUnitChord, decimals = 0))
#             print("printCostTran =")
#             print(np.around(npCostMatrixUnitTran, decimals = 0))
#             print("printCostAll =")
#             print(np.around(npCostMatrixUnit, decimals = 0))
            
            
        # debug
        if True:
            print("i=" + str(i))
            print("len possible[i] =" + str(len(possible[i])))
            print("shape npCostMatrixUnit = " + str(np.shape(npCostMatrixUnit)))
            print("shape npMinValue =" + str(np.shape(npMinValue)))
            print("shape npMinIndex =" + str(np.shape(npMinIndex)))
            print("shape npCombMinValue =" + str(np.shape(npCombMinValue)))
            print("len combMinIndex =" + str(len(combMinIndex)))
            print("shape preMinValue =" + str(np.shape(npPreMinValue)))
            print("len preMinIndex =" + str(len(preMinIndex)))
#             print("printCostAll =")
#             print(np.around(npCostMatrixUnit, decimals = 0))
#             print("printCostChord =")
#             print(np.around(npCostMatrixUnitChord, decimals = 0))

        # minValue 
        for imin in range(len(npMinValue)):
            npCombMinValue[imin] = npMinValue[imin] + npPreMinValue[npMinIndex[imin]]
            combMinIndex[imin] = copy.deepcopy(preMinIndex[npMinIndex[imin]])
            combMinIndex[imin].append(copy.deepcopy(minIndex[imin]))
        npPreMinValue = npCombMinValue
        preMinIndex = combMinIndex

        # debug
        if False:
            print("after cal with preValue")
            print("shape npMinValue =" + str(np.shape(npMinValue)))
            print("shape npMinIndex =" + str(np.shape(npMinIndex)))
            print("shape npCombMinValue =" + str(np.shape(npCombMinValue)))
            print("len combMinIndex =" + str(len(combMinIndex)))
            print("shape preMinValue =" + str(np.shape(npPreMinValue)))
            print("len preMinIndex =" + str(len(preMinIndex)))
            print("")
        # debug
        if True:
            print("npCombMinValue = " + str(npCombMinValue))
            print("combMinIndex =" + str(combMinIndex))
    #     minSummary = np.vstack
        costBest.append(copy.deepcopy([npCombMinValue,combMinIndex]))
    return costBest
# costBest[-1]

In [318]:
# debug
np.shape(npCostMatrixUnit)
np.shape(np.min(npCostMatrixUnit, axis = 0)    )
[[0] for i in range(iniSize)]
np.shape(np.argmin(npCostMatrixUnit, axis = 0))
np.shape(npCombMinValue)
np.shape(combMinIndex)
for i in possible:
    print(len(i))

13
24
38
38
27
20
20


In [69]:
# debug use, 計算特定chord之間的cost
mixPos0s =[ 
    [[5, 3, 4], [4, 2, 2], [2, 1, 1], [3, 0, 0], [1, 0, 0]],
    [[5, 3, 3], [4, 2, 2], [2, 1, 1], [3, 0, 0], [1, 0, 0]]
]
mixPos1 = [[4, 0, 0], [3, 2, 1], [2, 3, 3], [1, 2, 2]]
mixNotes0 = [48, 52, 60, 55, 64]
mixNotes1 = [50, 57, 62, 66]

for mixPos0 in mixPos0s:    
    print(mixPos0)
#     print(mixPos1)
#     print(mixNotes0)
#     print(mixNotes1)
    funcCalTranCostUnit(mixPos0, mixPos1,mixNotes0, mixNotes1)

[[5, 3, 4], [4, 2, 2], [2, 1, 1], [3, 0, 0], [1, 0, 0]]
ipos0 =0, ipos1 = 0, costAlong=0.6000000000000001
ipos0 =0, ipos1 = 1, costAlong=3.75
ipos0 =0, ipos1 = 2, costAlong=3.95
ipos0 =0, ipos1 = 3, costAlong=3.0
ipos0 =1, ipos1 = 0, costAlong=0.4
ipos0 =1, ipos1 = 1, costAlong=3.55
ipos0 =1, ipos1 = 2, costAlong=1.5
ipos0 =1, ipos1 = 3, costAlong=0.8
ipos0 =2, ipos1 = 0, costAlong=0.2
ipos0 =2, ipos1 = 1, costAlong=0.6000000000000001
ipos0 =2, ipos1 = 2, costAlong=1.3
ipos0 =2, ipos1 = 3, costAlong=1.1
ipos0 =3, ipos1 = 0, costAlong=0.0
ipos0 =3, ipos1 = 1, costAlong=0.4
ipos0 =3, ipos1 = 2, costAlong=0.6000000000000001
ipos0 =3, ipos1 = 3, costAlong=0.4
ipos0 =4, ipos1 = 0, costAlong=0.0
ipos0 =4, ipos1 = 1, costAlong=0.4
ipos0 =4, ipos1 = 2, costAlong=0.6000000000000001
ipos0 =4, ipos1 = 3, costAlong=0.4
costAlong = 23.55
costAcross = 6.5
[[5, 3, 3], [4, 2, 2], [2, 1, 1], [3, 0, 0], [1, 0, 0]]
ipos0 =0, ipos1 = 0, costAlong=0.6000000000000001
ipos0 =0, ipos1 = 1, costAlong=3.0
ipos0

In [62]:
# 這個Function 也可以用在melody to Mix, 不會被限制
# 他也許可以改為funcTranCostUnit, 專門比較transition的cost
def funcCalTranCostUnit(mixPos0, mixPos1,mixNotes0, mixNotes1, printOut = False):

    costTran = 0
    costAlongAll = 0
    costAcrossAll = 0
    costTiedNote = 0
    for ipos0 in range(len(mixPos0)):
#         print("ipos0=" + str(ipos0))
        for ipos1 in range(len(mixPos1)):
#             print("ipos1=" + str(ipos1))
            if mixNotes1[ipos1] < 0:
                costTiedNote = 0 # initialize tied-note
                try:
                    relativeNote0Index = mixNotes0.index(abs(mixNotes1[ipos1]))
                except:
                    relativeNote0Index = mixNotes0.index(   (mixNotes1[ipos1]))
#                 print("relativeNote0Index = " + str(relativeNote0Index))
                if mixPos1[ipos1] != mixPos0[ relativeNote0Index]:
#                     print("negative and !=")
                    costTiedNote = float("inf")
                    break
                # 可以走到這代表前次和這次的position相同
                # costAlong = 0, 所以不用做任何相加
            else:
                costAlong    = funcCalCostAlong(mixPos0[ipos0], mixPos1[ipos1])
                costAcross   = funcCalCostAcross(mixPos0[ipos0], mixPos1[ipos1])
#                 print("mixPos0[0] =" + str(mixPos0[0]))
#                 print("mixPos1[0] =" + str(mixPos1[0]))
#                 print("ipos0 =" + str(ipos0) + ", ipos1 = " + str(ipos1) + ", costAlong=" + str(costAlong))
#                 print("costAcross="+str(costAcross))
                # 先分開測測看
                costAlongAll = costAlongAll + costAlong
                costAcrossAll = costAcrossAll + costAcross
                #原本的是costTrain
#                 costTran = costTran + costAlong + costAcross
    ##### debug use #####
#     if mixPos0 == [[6, 3, 1], [2, 0, 0], [3, 0, 0]] and mixPos1 ==[[6, 3, 1], [4, 0, 0]] :
        
#     print("costAlong = " + str(costAlongAll))
#     print("costAcross = " + str(costAcrossAll))
    costTran = costAlongAll + costAcrossAll + costTiedNote

    costChord0   = funcCalChordCost(mixPos0)
    costChord1   = funcCalChordCost(mixPos1)
    costChord = (costChord0 + costChord1)* GlobalVar.get_costChordWeight()
#     costMixTotal = costMixTotal+ costChord
#     print(costChord)
    return costTran, costChord
        
# costMixTotal

In [10]:
# Calculate MIX transition cost

def funcCalTranCostMatrix(event0Possible, event1Possible, mixNotes0, mixNotes1):
    costMatrixUnitTran = []
    costMatrixUnitChord = []
    for mixPos0 in event0Possible:
        pos0beg =[]
        pos0begChord = []
        for mixPos1 in event1Possible:

            # cost is putting here
            costTran, costChord = funcCalTranCostUnit(mixPos0, mixPos1,mixNotes0, mixNotes1)
            
            # melody: highly dependent to Tran
            # chord: lower dependent to Tran
            if len(mixPos0) > 1 or len(mixPos1) > 1:
                costTran = costTran * 0.25

            pos0beg.append(costTran)
            pos0begChord.append(costChord)
        costMatrixUnitTran.append(pos0beg)
        costMatrixUnitChord.append(pos0begChord)
    return costMatrixUnitTran, costMatrixUnitChord

In [59]:
# Solutions
# 看要不要直接input costBest[-1][1]
# 裡面比較不用寫這麼複雜

def funcSolutions(costBest, possible, printOut=False):
    solutions = []

    print("Best paths of the last stage") if printOut else None
    for choose in range(len(costBest[-1][1])):
        bestPathNumber = costBest[-1][1][choose][1:]
        print(bestPathNumber)  if printOut else None
        bestPath = []
        for i in range(len(bestPathNumber)):
            bestPath.append(possible[i][bestPathNumber[i]])
        bestPath.append(possible[-1][choose])
        print("cost=" + '%.2f' % costBest[-1][0][choose] +" " + str(bestPath)) if printOut else None
        solutions.append(["cost =", costBest[-1][0][choose], bestPath ])
        
    choose = np.argmin(costBest[-1][0], axis = 0)
    bestSolution = solutions[choose]
    return solutions, bestSolution 

In [ ]:
看往後面第幾個event加上去的時候
會開始出現inf
要怎麼解(是否不要用inf來表示不可能的tran)

做成spyder會不會比較好debug